# Erdos Renyi Graphs
In this notebook we will work with random graphs generated using the Erdos Renyi model. These graphs will act as a representative sample of graphs with higher number of nodes to evaluate the performance of different isomorphism tests.

In [1]:
import networkx as nx
from tqdm import tqdm
from collections import defaultdict
from math import ceil

from counting import embed_graph
from weisfeiler_lehman import wl_embedding
from compare import compare_embeddings
from product.factor import get_factor_dict

from generator import generate_graph_product_table


## Graph Generation
We will generate random graphs of different sizes greater than 7 nodes (these will be inlcuded completely in the test set). The number of graphs will increase with the size of the graph, in order to try and have a more representative sample of the space of graphs. Then we will generate graphs with different edge densities, from 0.2 to 0.8. We create more graphs for densities closer to 0.5.

In [2]:
seed = 42 # global seed for reproducibility

def generate_erdos_renyi_graphs(count, n, p):
    global seed
    graphs = []
    progress = tqdm(total=count, desc=f"Generating Erdos-Renyi n:{n}, p:{p:.2f}")
    while len(graphs) < count:
        graph = nx.erdos_renyi_graph(n, p, seed=seed)
        seed += 1
        if nx.is_connected(graph) and not any([nx.is_isomorphic(graph, g) for g in graphs]):
            graphs.append(graph)
            progress.update(1)
    return graphs

In [3]:
counts = {
    8: 500,
    9: 600,
    10: 700,
    11: 800,
    12: 900,
}

sensitivities = [0.2, 0.4, 0.6, 0.8]

connected_graphs_leq_7 = [G for G in nx.graph_atlas_g() if not nx.is_empty(G) and nx.is_connected(G)]
er_graphs = defaultdict(list)

for G in connected_graphs_leq_7:
    er_graphs[G.number_of_nodes()].append(G)

for n, count in counts.items():
    for p in sensitivities:
        p_count = ceil(count * (1 - abs(p - 0.5)))
        new_er_graphs = generate_erdos_renyi_graphs(p_count, n, p)
        for g in new_er_graphs:
            if not any([nx.is_isomorphic(g, g_) for g_ in er_graphs[n]]):
                er_graphs[n].append(g)


Generating Erdos-Renyi n:12, p:0.80: 100%|██████████| 630/630 [00:04<00:00, 139.01it/s] 


In [7]:
er_graphs_list = [g for graphs in er_graphs.values() for g in graphs if g.number_of_nodes() <= 12]
len(er_graphs_list)

12042

## Isomorphism Tests
Next we will use basis cycle counting to create an embedding the generated graphs. Further we will apply graph products with different factor graphs to enhance the embeddings. Finally we will use the embeddings to compare the graphs and evaluate, how many collisions we have for graph pairs.

In [17]:
embed_config = {"size": 150, "operation": "basis_cycle"}
factors = get_factor_dict([3, 5, 7, 9], ["Path", "Star"])

print("Embedding graphs")
cycle_embedded_graphs = embed_graph(er_graphs_list, 150, "basis_cycle")
print("Embedding graph products")
cycle_embedded_products = generate_graph_product_table(er_graphs_list, products=["Strong", "Tensor", "Modular"], factors=factors, embedding=embed_config)

print("Comparing embeddings")
cycle_results = compare_embeddings(cycle_embedded_graphs, index=True)
cycle_product_results = cycle_embedded_products.map(lambda x: compare_embeddings(x, index=True))

print(f"Generated graphs without graph product application have {cycle_results[0]} collisions")

print("Cycle product results")
cycle_product_results.map(lambda x: x[0])

Embedding graphs
Embedding graph products
Comparing embeddings
Generated graphs without graph product application have 221861 collisions
Cycle product results


Graph Product,Strong,Tensor,Modular
Factor Graph,,,
P3,4684,54378,149
P5,203,1793,16
P7,66,489,16
P9,43,199,10
S3,8288,62420,523
S5,10249,46574,122
S7,6096,42212,104
S9,7077,42208,91


Now we will compare these results with isomorphism testing using Weisfeiler Lehman algorithm.

In [13]:
wl_embedded_graphs = wl_embedding(er_graphs_list, 10)
wl_results = compare_embeddings(wl_embedded_graphs, index=True)
print(f"Results: {wl_results[0]} collisions")

Results: 23 collisions


Now we will do the test separately for each graph size.

In [19]:
embed_config = {"size": 150, "operation": "basis_cycle"}
factors = get_factor_dict([3, 5, 7, 9], ["Path", "Star"])

for n, graphs in er_graphs.items():
    print(f"Graphs for n={n}")
    cycle_embedded_graphs = embed_graph(graphs, 150, "basis_cycle")
    cycle_embedded_products = generate_graph_product_table(graphs, products=["Strong", "Tensor", "Modular"], factors=factors, embedding=embed_config)

    cycle_results = compare_embeddings(cycle_embedded_graphs, index=True)
    cycle_product_results = cycle_embedded_products.map(lambda x: compare_embeddings(x, index=True))

    print(f"Generated graphs without graph product application have {cycle_results[0]} collisions")

    print("Cycle product results")
    print(cycle_product_results.map(lambda x: x[0]))

    wl_embedded_graphs = wl_embedding(graphs, 10)
    wl_results = compare_embeddings(wl_embedded_graphs, index=True)
    print(f"WL-Comparison Results: {wl_results[0]} collisions")
    print("-" * 80)

Graphs for n=2
Generated graphs without graph product application have 0 collisions
Cycle product results
Graph Product  Strong  Tensor  Modular
Factor Graph                          
P3                  0       0        0
P5                  0       0        0
P7                  0       0        0
P9                  0       0        0
S3                  0       0        0
S5                  0       0        0
S7                  0       0        0
S9                  0       0        0
WL-Comparison Results: 0 collisions
--------------------------------------------------------------------------------
Graphs for n=3
Generated graphs without graph product application have 0 collisions
Cycle product results
Graph Product  Strong  Tensor  Modular
Factor Graph                          
P3                  0       0        0
P5                  0       0        0
P7                  0       0        0
P9                  0       0        0
S3                  0       0        0
S5      